In [1]:
import io
from ipyfileupload.widgets import DirectoryUploadWidget
def _upload():

    _upload_widget = DirectoryUploadWidget()

    def _cb(change):
        for name, file in change['owner'].files.items():
            decoded = io.StringIO(file.decode('utf-8'))
            print('Uploaded `{}` ({:.2f} kB)'.format(
                name, len(decoded.read()) / 2 **10))

    _upload_widget.observe(_cb, names='base64_files')
    display(_upload_widget)
    return _upload_widget

widget = _upload()

DirectoryUploadWidget(label='Browse', _dom_classes=('widget_item', 'btn-group'))

In [2]:
from classes.ipywidgets import *
from classes.Titration import Titration, TitrationCLI
from ipyfileupload.widgets import DirectoryUploadWidget

TitrationWidget.titration = Titration()
DirectoryUploadWidget()

DirectoryUploadWidget(label='Browse', _dom_classes=('widget_item', 'btn-group'))

In [3]:


view = TitrationFilesView()
upload = TitrationDirUploader()
upload.add_observer(view)
display(
    upload,
    view
)

TitrationDirUploader(label='Upload titration directory...', style=ButtonStyle(), _dom_classes=('widget_item', 'btn-group'))

TitrationFilesView(children=(VBox(children=(Label(value='Data files', _dom_classes=('panel-header-title',)),), _dom_classes=('panel-heading',)), VBox(_dom_classes=('panel-body',))), layout=Layout(width='250px'), _dom_classes=('panel', 'panel-default', 'data-files-view'))

In [4]:
startParams = StartParamContainer()
protocole = ProtocoleContainer()

upload.add_observer(startParams)
upload.add_observer(protocole)

startParams.add_observer(protocole.protocole)

display(startParams, protocole)

StartParamContainer(children=(VBox(children=(NameWidget(value='UIM-SH3 Domain', description='Experiment name:', layout=Layout(width='335px'), style=DescriptionStyle(description_width='130px'), _dom_classes=('panel-header-title',)),), _dom_classes=('panel-heading',)), VBox(children=(MoleculesContainer(children=(SingleMolContainer(children=(Label(value='Analyte', layout=Layout(left='150px')), MoleculeNameWidget(value='UIM-SH3', description='Name:', layout=Layout(width='230px'), style=DescriptionStyle(description_width='130px')), ConcentrationWidget(value=375.0, description='Concentration (µM):', layout=Layout(width='230px'), max=100000.0, style=DescriptionStyle(description_width='130px')))), SingleMolContainer(children=(Label(value='Titrant'), MoleculeNameWidget(value='pep', layout=Layout(width='100px')), ConcentrationWidget(value=2000.0, layout=Layout(width='100px'), max=100000.0))))), FloatControlWidget(value=45.0, description='Analyte volume (µL):', layout=Layout(width='230px'), max=100000.0, style=DescriptionStyle(description_width='130px')), FloatControlWidget(value=200.0, description='Total volume (µL):', layout=Layout(width='230px'), max=100000.0, style=DescriptionStyle(description_width='130px'))), _dom_classes=('panel-body',))), layout=Layout(width='400px'), _dom_classes=('panel', 'panel-default'))

ProtocoleContainer(children=(VolumePanel(children=(VBox(children=(Label(value='Added titrant volumes', _dom_classes=('panel-header-title',)), HBox(children=(Button(button_style='primary', description='Add', icon='plus-circle', layout=Layout(width='50%'), style=ButtonStyle()), Button(button_style='warning', description='Remove', icon='minus-circle', layout=Layout(width='50%'), style=ButtonStyle())), layout=Layout(padding='5px 5px 5px 5px'))), _dom_classes=('panel-heading',)), VBox(children=(VBox(children=(VolumeWidget(value=0.0, description='Step 0', disabled=True, layout=Layout(height='28px', width='100%'), max=100000.0, style=DescriptionStyle(description_width='50%')), VolumeWidget(value=2.0, description='Step 1', layout=Layout(height='28px', width='100%'), max=100000.0, style=DescriptionStyle(description_width='50%')), VolumeWidget(value=2.0, description='Step 2', layout=Layout(height='28px', width='100%'), max=100000.0, style=DescriptionStyle(description_width='50%')), VolumeWidget(value=4.0, description='Step 3', layout=Layout(height='28px', width='100%'), max=100000.0, style=DescriptionStyle(description_width='50%')), VolumeWidget(value=4.0, description='Step 4', layout=Layout(height='28px', width='100%'), max=100000.0, style=DescriptionStyle(description_width='50%')), VolumeWidget(value=4.0, description='Step 5', layout=Layout(height='28px', width='100%'), max=100000.0, style=DescriptionStyle(description_width='50%')), VolumeWidget(value=5.0, description='Step 6', layout=Layout(height='28px', width='100%'), max=100000.0, style=DescriptionStyle(description_width='50%')), VolumeWidget(value=8.0, description='Step 7', layout=Layout(height='28px', width='100%'), max=100000.0, style=DescriptionStyle(description_width='50%')), VolumeWidget(value=10.0, description='Step 8', layout=Layout(height='28px', width='100%'), max=100000.0, style=DescriptionStyle(description_width='50%')), VolumeWidget(value=10.0, description='Step 9', layout=Layout(height='28px', width='100%'), max=100000.0, style=DescriptionStyle(description_width='50%')), VolumeWidget(value=10.0, description='Step 10', layout=Layout(height='28px', width='100%'), max=100000.0, style=DescriptionStyle(description_width='50%'))), layout=Layout(display='flex', flex_flow='column', padding='5px 5px 5px 5px')),), _dom_classes=('panel-body',)), VBox(children=(Button(button_style='success', description='Validate', icon='check', layout=Layout(width='100%'), style=ButtonStyle()),), _dom_classes=('panel-footer',))), layout=Layout(height='auto', width='250px'), _dom_classes=('panel', 'panel-default', 'volumes-container')), ProtocolePanel(children=(VBox(children=(Label(value='Protocole', _dom_classes=('panel-header-title',)),), _dom_classes=('panel-heading',)), VBox(children=(HTML(value='<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Step</th>\n      <th>Added pep (µL)</th>\n      <th>Total pep (µL)</th>\n      <th>Total volume (µL)</th>\n      <th>[pep] (µM)</th>\n      <th>[UIM-SH3] (µM)</th>\n      <th>[pep]/[UIM-SH3]</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>0</td>\n      <td>0.0</td>\n      <td>0.0</td>\n      <td>200.0</td>\n      <td>0.000</td>\n      <td>84.375</td>\n      <td>0.000</td>\n    </tr>\n    <tr>\n      <td>1</td>\n      <td>2.0</td>\n      <td>2.0</td>\n      <td>202.0</td>\n      <td>19.802</td>\n      <td>83.540</td>\n      <td>0.237</td>\n    </tr>\n    <tr>\n      <td>2</td>\n      <td>2.0</td>\n      <td>4.0</td>\n      <td>204.0</td>\n      <td>39.216</td>\n      <td>82.721</td>\n      <td>0.474</td>\n    </tr>\n    <tr>\n      <td>3</td>\n      <td>4.0</td>\n      <td>8.0</td>\n      <td>208.0</td>\n      <td>76.923</td>\n      <td>81.130</td>\n      <td>0.948</td>\n    </tr>\n    <tr>\n      <td>4</td>\n      <td>4.0</td>\n      <td>12.0</td>\n      <td>212.0</td>\n      <td>113.208</td>\n      <td>79.599</td>\n      <td>1.422</td>\n    </tr>\n    <tr>\n      <td>5</td>\n      <td>4.0</td

In [5]:
import bqplot
import math 
from itertools import chain
from ipywidgets import FloatSlider, IntSlider, ToggleButton
from classes.bqplotwidgets import IntensityPanel
#HBox([CutoffSlider(),IntensityBarPlot()])
plotc = IntensityPanel()
display(plotc)

IntensityPanel(children=(VBox(children=(Label(value='Chemical shift intensities', _dom_classes=('panel-header-title',)),), _dom_classes=('panel-heading',)), VBox(children=(IntensityPlot(children=(IntSlider(value=10, description='Step', layout=Layout(height='95%', padding='30px 0px 30px 0px'), max=10, min=1, orientation='vertical'), CutoffSlider(value=0.1, description='Cut-off', layout=Layout(height='95%', padding='30px 0px 30px 0px'), max=1.7487105999564347, orientation='vertical', step=0.01), IntensityBarPlot(axes=[Axis(grid_lines='none', label='Residues', num_ticks=11, scale=OrdinalScale()), Axis(grid_color='#D1D0CE', grid_lines='dashed', orientation='vertical', scale=LinearScale(max=1.7487105999564347, min=0.0), tick_format='0.2f')], fig_margin={'top': 60, 'left': 60, 'right': 60, 'bottom': 60}, layout=Layout(height='95%', width='100%'), marks=[Bars(colors=['#38ACEC'], interactions={'click': 'select', 'legend_hover': 'highlight_axes', 'hover': 'tooltip'}, scales={'y': LinearScale(max=1.7487105999564347, min=0.0), 'x': OrdinalScale()}, scales_metadata={'y': {'dimension': 'y', 'orientation': 'vertical'}, 'x': {'dimension': 'x', 'orientation': 'horizontal'}, 'color': {'dimension': 'color'}}, selected=[0, 1, 2, 3, 4, 7, 8, 39, 41, 67, 68, 70, 73, 74, 75, 76, 80, 82, 85, 87, 91, 92, 94, 95, 99, 100], selected_style={'stroke': 'white', 'fill': 'orange'}, stroke='#FFFFFF', tooltip=Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['Residue', 'Intensity']), tooltip_style={'opacity': 0.9}, x=array([166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178,
       179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191,
       192, 193, 194, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205,
       206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218,
       219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231,
       232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244,
       245, 246, 247, 248, 249, 250, 251, 253, 254, 255, 256, 257, 258,
       260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272]), y=array([  3.96001263e-01,   1.34078335e-01,   2.06011747e-01,
         4.51144057e-01,   1.04000769e-01,   2.90006896e-02,
         6.79411510e-03,   1.07026913e-01,   1.04000769e-01,
         5.00399840e-03,   4.40549657e-02,   5.01022954e-02,
         6.86466314e-02,   4.40549657e-02,   5.01597448e-03,
         1.90168346e-02,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   1.14017543e-02,
         0.00000000e+00,   1.14017543e-02,   2.00000000e-03,
         1.50119952e-02,   2.40000000e-02,   5.00399840e-03,
         1.00000000e-03,   6.01331855e-03,   2.90006896e-02,
         3.40052937e-02,   2.03960781e-03,   4.30074412e-02,
         2.40000000e-02,   5.03587132e-03,   3.02654919e-03,
         2.90006896e-02,   2.60007692e-02,   1.50119952e-02,
         2.40024082e-01,   1.50119952e-02,   1.79069819e-01,
         7.85875308e-03,   0.00000000e+00,   3.40052937e-02,
         9.10562464e-02,   1.70011764e-02,   0.00000000e+00,
         6.80073525e-02,   2.03960781e-03,   0.00000000e+00,
         0.00000000e+00,   7.50095994e-02,   9.00888450e-03,
         8.00249961e-03,   4.50111097e-02,   0.00000000e+00,
         0.00000000e+00,   2.40000000e-02,   3.60005556e-02,
         2.40008333e-02,   5.80675469e-02,   1.14017543e-02,
         6.10209800e-02,   5.70014035e-02,   2.23606798e-02,
         9.90202000e-02,   5.08566613e-01,   4.90467084e-01,
         3.70345784e-02,   5.12516692e-01,   8.21671467e-02,
         5.15270803e-02,   1.58579083e+00,   1.95473783e-01,
         3.26675925e-01,   3.61354397e-01,   7.62758153e-02,
         9.70000000e-02,   2.44131112e-03,   1.29000155e-01,
         4.35343543e-02,   1.12171476e-01,   6.50249183e-02,
         5.50058179e-02,   1.17008376e-01,   6.00962561e-02,
         1.33005413e-01,   8.59883713e-02,   4.61406545e-02,
         9.10711810e-02,   1.13553688e-01,   1.14092769e-01,
         1.14

In [93]:
import numpy as np
class ShiftMap(bqplot.Figure, TitrationWidget):
    
    def __init__(self, *args, **kwargs):
        
        self.x_scale = bqplot.LinearScale()
        self.y_scale = bqplot.LinearScale()
        self.c_scale = bqplot.ColorScale(scheme='Purples')
        
        self.ax_x = bqplot.Axis(
            label="Proton chemical shifts", 
            scale=self.x_scale,
            tick_format='0.2f',
            grid_lines='dashed',
            grid_color='#D1D0CE',
            num_ticks=10
            #visible = not stacked
        )
        
        #
        
        self.ax_y = bqplot.Axis(
            label="Azote chemical shifts", 
            scale=self.y_scale, 
            orientation='vertical', 
            tick_format='0.2f',
            grid_lines='dashed',
            grid_color='#D1D0CE',
            num_ticks=10
            #visible = not stacked
        )
        
        self.ax_c = bqplot.ColorAxis(
            label_location='middle', 
            scale=self.c_scale, 
            orientation='vertical', 
            side='right',
            label="Step",
            #label_color = 'blue',
            offset = {'scale':self.x_scale,'value':0}
        )
        
        residues = self.titration.filtered.values()
        
        self.x_data = []
        self.y_data = []
        self.c_data = []
        
        for res in residues:
            self.x_data += res.chemshiftH
            self.y_data += res.chemshiftN
            self.c_data += list(range(self.titration.dataSteps))
            
        self.scatter = bqplot.Scatter(
            x=self.x_data, 
            y=self.y_data, 
            color=self.c_data,
            marker='circle',
            stroke='#FFFFFF',
            #default_opacities = np.linspace(0.1, 0.8, num=self.titration.dataSteps).tolist(),
            stroke_width = 0,
            scales= {'x': self.x_scale, 'y': self.y_scale, 'color':self.c_scale},
            selected_style={'stroke': 'white', 'fill': 'orange'},
            interactions = {
                'legend_hover': 'highlight_axes',
                'hover': 'tooltip', 
                #'click': 'select',
            }
        )

        self.set_tooltips()
        
        bqplot.Figure.__init__(
            self, 
            marks=[self.scatter], 
            axes=[self.ax_x, self.ax_y, self.ax_c],
            title="Chemical shifts",
            *args, **kwargs)
        
        self.layout=Layout(width='100%', height='width')
        
    def set_tooltips(self):
        return
        # Adding a tooltip on hover in addition to select on click
        def_tt = bqplot.Tooltip(
            fields=['x', 'y'], 
            formats=['', '.2f'],
            labels=('Residue', 'Intensity'))
        self.bar_chart.tooltip=def_tt
        self.bar_chart.interactions = {
            'legend_hover': 'highlight_axes',
            'hover': 'tooltip', 
            'click': 'select',
        }
        
ShiftMap()

ShiftMap(axes=[Axis(grid_color='#D1D0CE', grid_lines='dashed', label='Proton chemical shifts', num_ticks=10, scale=LinearScale(), tick_format='0.2f'), Axis(grid_color='#D1D0CE', grid_lines='dashed', label='Azote chemical shifts', num_ticks=10, orientation='vertical', scale=LinearScale(), tick_format='0.2f'), ColorAxis(label='Step', offset={'scale': LinearScale(), 'value': 0}, orientation='vertical', scale=ColorScale(scheme='Purples'), side='right')], fig_margin={'top': 60, 'left': 60, 'right': 60, 'bottom': 60}, layout=Layout(height='width', width='100%'), marks=[Scatter(color=array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,
        6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,
        1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,
        7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,
        2,  3,  4,  5,  6,  7,  8,  9, 10]), colors=['DeepSkyBlue'], interactions={'legend_hover': 'highlight_axes', 'hover': 'tooltip'}, scales={'y': LinearScale(), 'x': LinearScale(), 'color': ColorScale(scheme='Purples')}, scales_metadata={'skew': {'dimension': 'skew'}, 'y': {'dimension': 'y', 'orientation': 'vertical'}, 'x': {'dimension': 'x', 'orientation': 'horizontal'}, 'color': {'dimension': 'color'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'size': {'dimension': 'size'}}, selected_style={'stroke': 'white', 'fill': 'orange'}, stroke='#FFFFFF', stroke_width=0.0, tooltip_style={'opacity': 0.9}, x=array([ 112.069,  112.035,  111.903,  111.867,  111.705,  111.676,
        111.709,  111.692,  111.632,  111.645,  111.673,  119.205,
        119.209,  119.209,  119.209,  117.82 ,  117.808,  117.734,
        117.65 ,  117.626,  117.622,  117.626,  119.715,  119.691,
        119.596,  119.563,  119.393,  119.391,  119.368,  119.351,
        119.351,  119.355,  119.354,  121.784,  121.765,  121.709,
        121.689,  121.368,  121.386,  121.372,  121.331,  121.321,
        121.311,  121.333,  113.731,  113.698,  113.41 ,  113.318,
        113.287,  113.253,  113.246,  113.226,  113.223,  113.214,
        113.223,  118.099,  118.102,  118.279,  118.34 ,  118.505,
        118.511,  118.515,  118.547,  118.586,  118.566,  118.589,
        127.624,  127.589,  127.345,  127.356,  127.188,  127.165,
        127.133,  127.123,  127.117,  127.106,  127.113]), y=array([ 8.999,  8.999,  9.   ,  9.   ,  9.003,  9.002,  9.004,  9.005,
        9.003,  9.004,  9.004,  7.672,  7.662,  7.662,  7.662,  7.031,
        7.016,  6.981,  6.961,  6.948,  6.941,  6.939,  7.495,  7.498,
        7.519,  7.533,  7.57 ,  7.569,  7.574,  7.577,  7.577,  7.576,
        7.575,  7.747,  7.749,  7.757,  7.757,  7.798,  7.799,  7.801,
        7.802,  7.804,  7.803,  7.804,  6.992,  7.   ,  7.057,  7.076,
        7.096,  7.101,  7.107,  7.111,  7.113,  7.113,  7.112,  8.435,
        8.438,  8.474,  8.475,  8.527,  8.531,  8.535,  8.54 ,  8.544,
        8.545,  8.542,  7.586,  7.581,  7.486,  7.482,  7.418,  7.413,
        7.399,  7.394,  7.39 ,  7.388,  7.389]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0), title='Chemical shifts')

In [15]:
print(TitrationWidget.titration.filtered)

{256: (256, [112.069, 112.035, 111.903, 111.867, 111.705, 111.676, 111.709, 111.692, 111.632, 111.645, 111.673], [8.999, 8.999, 9.0, 9.0, 9.003, 9.002, 9.004, 9.005, 9.003, 9.004, 9.004]), 224: (224, [119.205, 119.209, 119.209, 119.209, 117.82, 117.808, 117.734, 117.65, 117.626, 117.622, 117.626], [7.672, 7.662, 7.662, 7.662, 7.031, 7.016, 6.981, 6.961, 6.948, 6.941, 6.939]), 260: (260, [121.784, 121.765, 121.709, 121.689, 121.368, 121.386, 121.372, 121.331, 121.321, 121.311, 121.333], [7.747, 7.749, 7.757, 7.757, 7.798, 7.799, 7.801, 7.802, 7.804, 7.803, 7.804]), 218: (218, [113.731, 113.698, 113.41, 113.318, 113.287, 113.253, 113.246, 113.226, 113.223, 113.214, 113.223], [6.992, 7.0, 7.057, 7.076, 7.096, 7.101, 7.107, 7.111, 7.113, 7.113, 7.112]), 219: (219, [118.099, 118.102, 118.279, 118.34, 118.505, 118.511, 118.515, 118.547, 118.586, 118.566, 118.589], [8.435, 8.438, 8.474, 8.475, 8.527, 8.531, 8.535, 8.54, 8.544, 8.545, 8.542]), 221: (221, [127.624, 127.589, 127.345, 127.356, 12